In [1]:
import boto3
import pandas as pd
import numpy as np
import re
from io import StringIO
import datetime
from dotenv import load_dotenv


In [2]:
# Nazwa bucketu S3
BUCKET_NAME = "zadmod-9"

# Wczytanie zmiennych środowiskowych
load_dotenv()

# Inicjalizacja klienta S3
s3 = boto3.client("s3")

def load_data_from_s3(file_key):
    """Wczytuje dane z S3 do DataFrame."""
    obj = s3.get_object(Bucket=BUCKET_NAME, Key=file_key)
    csv_data = obj['Body'].read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_data), sep=";")
    return df

# Wczytaj dane dla 2023 i 2024
wroclaw_2023_df = load_data_from_s3("Dane_mod9/halfmarathon_wroclaw_2023__final.csv")
wroclaw_2024_df = load_data_from_s3("Dane_mod9/halfmarathon_wroclaw_2024__final.csv")


In [3]:
# I. KONWERSJA CZASU
def convert_time_to_seconds(time):
    if pd.isnull(time) or time in ['DNS', 'DNF']: #DID NOT START / DID NOT FINISH
        return None
    time = time.split(':')
    return int(time[0]) * 3600 + int(time[1]) * 60 + int(time[2])


In [4]:
import pandas as pd

def get_cleaned_dataframe(df_2023: pd.DataFrame, df_2024: pd.DataFrame) -> pd.DataFrame:
    df_combined = pd.concat([df_2023, df_2024], ignore_index=True)  # Łączenie danych z obu lat
    df_model = df_combined[df_combined['Miejsce'].notnull()].copy() # Usuwanie wierszy z brakującymi miejscami 

    # Usuwanie zbędnych kolumn
    columns_to_drop = ['Drużyna', 'Miasto'] # Wybrane kolumny do usunięcia
    df_model.drop(columns=columns_to_drop, inplace=True)

    return df_model


In [5]:
cleaned_df = get_cleaned_dataframe(wroclaw_2023_df, wroclaw_2024_df)
missing_values_count = cleaned_df.isna().sum().reset_index(name='ilość')


In [6]:
cleaned_df.isna().sum().reset_index(name='ilość')


,index,ilość
0,Miejsce,0
1,Numer startowy,0
2,Imię,0
3,Nazwisko,0
4,Kraj,0
5,Płeć,0
6,Płeć Miejsce,0
7,Kategoria wiekowa,20
8,Kategoria wiekowa Miejsce,20
9,Rocznik,485


In [7]:
def impute_rocznik_from_category(df: pd.DataFrame) -> pd.DataFrame:
    """
    Uzupełnia brakujące wartości w kolumnie 'Rocznik'
    medianą dla każdej 'Kategorii wiekowej' i dodaje kolumnę 'Wiek'.
    """
    if 'Rocznik' not in df.columns or 'Kategoria wiekowa' not in df.columns:
        return df

    df_result = df.copy()

    # Oblicz medianę 'Rocznik' dla każdej kategorii wiekowej
    median_map = df_result.groupby('Kategoria wiekowa')['Rocznik'].transform('median').round(0).astype('Int64')

    # Uzupełnij brakujące 'Rocznik' medianą odpowiedniej kategorii
    df_result['Rocznik'] = df_result['Rocznik'].fillna(median_map)

    # Dodaj kolumnę 'Wiek'
    df_result['Wiek'] = 2024 - df_result['Rocznik']

    # Usuń kolumnę 'Rocznik'
    df_result = df_result.drop(columns=['Rocznik'])

    # Wyświetl liczby brakujących przed i po
    print(f"Brakujące wartości w 'Rocznik' przed imputacją: {(df['Rocznik'].isnull().sum())}")
    print(f"Brakujące wartości w 'Rocznik' po imputacji: {(df_result['Wiek'].isnull().sum())}")

    return df_result


In [8]:
cleaned_df = get_cleaned_dataframe(wroclaw_2023_df, wroclaw_2024_df)
cleaned_df = impute_rocznik_from_category(cleaned_df)
cleaned_df


Brakujące wartości w 'Rocznik' przed imputacją: 485
Brakujące wartości w 'Rocznik' po imputacji: 0


,Miejsce,Numer startowy,Imię,Nazwisko,Kraj,Płeć,Płeć Miejsce,Kategoria wiekowa,Kategoria wiekowa Miejsce,5 km Czas,...,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo,Wiek
0,1.0,1787,TOMASZ,GRYCKO,POL,M,1.0,M30,1.0,00:14:37,...,00:44:47,1.0,3.106667,01:01:43,1.0,3.386667,0.031400,01:04:59,3.080509,32.0
1,2.0,3,ARKADIUSZ,GARDZIELEWSKI,POL,M,2.0,M30,2.0,00:14:48,...,00:45:26,2.0,3.143333,01:03:08,2.0,3.540000,0.038000,01:06:23,3.146875,38.0
2,3.0,3832,KRZYSZTOF,HADAS,POL,M,3.0,M20,1.0,00:15:46,...,00:47:34,3.0,3.236667,01:05:09,3.0,3.516667,0.024067,01:08:24,3.242475,28.0
3,4.0,416,DAMIAN,DYDUCH,POL,M,4.0,M30,3.0,00:16:11,...,00:48:49,5.0,3.330000,01:06:54,4.0,3.616667,0.025467,01:10:16,3.330963,36.0
4,5.0,8476,KAMIL,MAŃKOWSKI,POL,M,5.0,M20,2.0,00:16:12,...,00:49:31,7.0,3.386667,01:07:27,5.0,3.586667,0.023000,01:10:27,3.339654,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19245,10298.0,898,EWA KATARZYNA,KASIERSKA,POL,K,3059.0,K70,6.0,00:43:30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03:19:47,9.470649,76.0
19246,10299.0,10456,MAGDALENA,PRYNDA,POL,K,3060.0,K40,1030.0,00:42:06,...,02:15:28,10301.0,9.580000,03:10:19,10303.0,10.970000,0.162733,03:21:08,9.534645,46.0
19247,10300.0,11311,HELENA,KOLANY,POL,K,3061.0,K60,48.0,00:49:20,...,02:23:22,10303.0,9.010000,03:11:34,10304.0,9.640000,-0.029333,03:22:18,9.589950,69.0
19248,10301.0,8404,JOANNA,KAJDANEK,POL,K,3062.0,K40,1031.0,00:46:48,...,02:26:03,10304.0,9.980000,03:17:25,10305.0,10.273333,0.057000,03:28:32,9.885439,43.0


In [9]:
# Zmieniasz kolumny czasu w cleaned_df
time_columns = ['5 km Czas', '10 km Czas', '15 km Czas', '20 km Czas', 'Czas']
cleaned_df[time_columns] = cleaned_df[time_columns].applymap(convert_time_to_seconds)

# Teraz wyświetlasz
print(cleaned_df)


       Miejsce  Numer startowy           Imię       Nazwisko Kraj Płeć  \
0          1.0            1787         TOMASZ         GRYCKO  POL    M   
1          2.0               3      ARKADIUSZ  GARDZIELEWSKI  POL    M   
2          3.0            3832      KRZYSZTOF          HADAS  POL    M   
3          4.0             416         DAMIAN         DYDUCH  POL    M   
4          5.0            8476          KAMIL      MAŃKOWSKI  POL    M   
...        ...             ...            ...            ...  ...  ...   
19245  10298.0             898  EWA KATARZYNA      KASIERSKA  POL    K   
19246  10299.0           10456      MAGDALENA         PRYNDA  POL    K   
19247  10300.0           11311         HELENA         KOLANY  POL    K   
19248  10301.0            8404         JOANNA       KAJDANEK  POL    K   
19249  10302.0           11155      Anonimowy       ZAWODNIK  POL    M   

       Płeć Miejsce Kategoria wiekowa  Kategoria wiekowa Miejsce  5 km Czas  \
0               1.0             

C:\Users\Michal\AppData\Local\Temp\ipykernel_18976\3704622056.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_df[time_columns] = cleaned_df[time_columns].applymap(convert_time_to_seconds)


In [10]:
cleaned_df.dropna(subset=time_columns, inplace=True)


In [11]:
cleaned_df.isna().sum().reset_index(name='ilość')


,index,ilość
0,Miejsce,0
1,Numer startowy,0
2,Imię,0
3,Nazwisko,0
4,Kraj,0
5,Płeć,0
6,Płeć Miejsce,0
7,Kategoria wiekowa,17
8,Kategoria wiekowa Miejsce,17
9,5 km Czas,0


In [12]:
# Funkcja do policzenia liczby outliers w serii
def count_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = series[(series < lower_bound) | (series > upper_bound)]
    return len(outliers)

# Funkcja do usuwania outliers z DataFrame w podanej kolumnie
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)].copy()

# Lista kolumn, z których chcesz usunąć outliers
columns_to_clean = [
    'Tempo', 
    '5 km Czas', '10 km Czas', '15 km Czas', '20 km Czas', 'Czas',
    '5 km Tempo', '10 km Tempo', '15 km Tempo', '20 km Tempo'
]

# Liczba outliers przed usunięciem
outliers_counts = {}
for col in columns_to_clean:
    if col in cleaned_df.columns and pd.api.types.is_numeric_dtype(cleaned_df[col]):
        outliers_counts[col] = count_outliers(cleaned_df[col])

print("--- Liczba outliers przed usunięciem ---")
for col, count in outliers_counts.items():
    print(f"{col}: {count}")

# Usuwanie outliers kaskadowo
for col in columns_to_clean:
    if col in cleaned_df.columns and pd.api.types.is_numeric_dtype(cleaned_df[col]):
        cleaned_df = remove_outliers(cleaned_df, col)

print(f"\nLiczba rekordów po usunięciu outliers: {len(cleaned_df)}")


--- Liczba outliers przed usunięciem ---
Tempo: 222
5 km Czas: 135
10 km Czas: 178
15 km Czas: 203
20 km Czas: 225
Czas: 222
5 km Tempo: 135
10 km Tempo: 256
15 km Tempo: 294
20 km Tempo: 408

Liczba rekordów po usunięciu outliers: 17626


In [13]:
cleaned_df


,Miejsce,Numer startowy,Imię,Nazwisko,Kraj,Płeć,Płeć Miejsce,Kategoria wiekowa,Kategoria wiekowa Miejsce,5 km Czas,...,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo,Wiek
8,9.0,5657,CYPRIAN,GRZELKA,POL,M,9.0,M20,4.0,1030.0,...,3131.0,12.0,3.560000,4263.0,10.0,3.773333,0.022733,4456,3.520581,23.0
10,11.0,7270,MATEUSZ,POŁCZYŃSKI,POL,M,11.0,M20,5.0,1029.0,...,3131.0,13.0,3.560000,4282.0,11.0,3.836667,0.026667,4483,3.541914,27.0
11,12.0,4367,MACIEJ,KARDAS,POL,M,12.0,M30,6.0,1035.0,...,3133.0,16.0,3.566667,4284.0,12.0,3.836667,0.026000,4490,3.547444,32.0
12,13.0,923,SEBASTIAN,CIEŚLA,POL,M,13.0,M20,6.0,1035.0,...,3133.0,14.0,3.566667,4290.0,13.0,3.856667,0.027200,4496,3.552185,28.0
13,14.0,4565,WOJCIECH,DURCZYŃSKI,POL,M,14.0,M30,7.0,1036.0,...,3134.0,17.0,3.566667,4292.0,14.0,3.860000,0.027200,4505,3.559295,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18999,10052.0,3158,PRZEMYSŁAW,STELMACH,POL,M,7129.0,M40,2360.0,2219.0,...,6975.0,10070.0,8.166667,9593.0,10060.0,8.726667,0.089400,10194,8.054041,41.0
19000,10053.0,1987,PAWEŁ,NOWICKI,POL,M,7130.0,M40,2361.0,2220.0,...,6981.0,10076.0,8.186667,9592.0,10057.0,8.703333,0.088267,10194,8.054041,41.0
19003,10056.0,11600,KATARZYNA,KOPROWSKA,POL,K,2925.0,K30,1013.0,2289.0,...,6958.0,10062.0,8.040000,9593.0,10059.0,8.783333,0.079533,10206,8.063522,38.0
19008,10061.0,9801,IWONA,HOŁOWACZ,POL,K,2928.0,K40,990.0,2146.0,...,6877.0,10029.0,8.243333,9594.0,10062.0,9.056667,0.128533,10218,8.073003,49.0


In [14]:
from pycaret.regression import setup, compare_models, finalize_model, plot_model, save_model, predict_model


In [15]:
# Eksperymwnt na całymm zbiorze z zmienną docelową 'Czas'
exp = setup(
    data=cleaned_df,
    target='Czas', # Czas ukończenia biegu w sekundach jako zmienna docelowa
    session_id=123,
    verbose=True,
    
)
# exp.dataset_transformed.sample(10)

,Description,Value
0,Session id,123
1,Target,Czas
2,Target type,Regression
3,Original data shape,"(17626, 25)"
4,Transformed data shape,"(17626, 37)"
5,Transformed train set shape,"(12338, 37)"
6,Transformed test set shape,"(5288, 37)"
7,Numeric features,19
8,Categorical features,5
9,Rows with missing values,0.1%


In [16]:
# Trenuje i ocenia wszystkie modele za pomocą walidacji krzyżowej (domyślnie 10-krotnej).
best_model_v1 = exp.compare_models(sort='MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,0.4527,5.6846,1.8766,1.0000,0.0003,0.0001,1.6840
dt,Decision Tree Regressor,0.7493,8.8799,2.7267,1.0000,0.0004,0.0001,0.0850
et,Extra Trees Regressor,0.9715,16.2555,3.6094,1.0000,0.0005,0.0001,0.7880
lightgbm,Light Gradient Boosting Machine,4.9186,71.5053,8.3454,0.9999,0.0012,0.0007,0.2070
gbr,Gradient Boosting Regressor,7.7955,110.0103,10.4792,0.9999,0.0015,0.0011,0.7800
huber,Huber Regressor,23.5257,1171.9758,34.1889,0.9990,0.0043,0.0031,0.4080
par,Passive Aggressive Regressor,28.7686,1426.1454,37.5555,0.9988,0.0049,0.0039,0.5310
ada,AdaBoost Regressor,53.8290,4783.4001,69.0416,0.9959,0.0097,0.0075,0.4000
dummy,Dummy Regressor,870.0064,1158167.7093,1075.9946,-0.0012,0.1494,0.1230,0.0590


In [17]:
# Ignoruje kolumny z tempem, ponieważ nie mają ścisłe powiązanie z czasem
exp = setup(
    data=cleaned_df,
    target='Czas', # Czas ukończenia biegu w sekundach jako zmienna docelowa
    ignore_features=['Tempo', '5 km Tempo', '10 km Tempo', '15 km Tempo', '20 km Tempo'],
    keep_features=['20 km Czas', '15 km Czas', '10 km Czas', '5 km Czas','Wiek','Płeć'],
    session_id=123,
    verbose=True,
    
)
# exp.dataset_transformed.sample(10)


,Description,Value
0,Session id,123
1,Target,Czas
2,Target type,Regression
3,Original data shape,"(17626, 25)"
4,Transformed data shape,"(17626, 32)"
5,Transformed train set shape,"(12338, 32)"
6,Transformed test set shape,"(5288, 32)"
7,Ignore features,5
8,Numeric features,14
9,Categorical features,5


In [18]:
best_model_v2 = exp.compare_models(sort='MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,3.4997,88.4462,9.0353,0.9999,0.0012,0.0005,0.7700
rf,Random Forest Regressor,3.5419,113.8365,10.2224,0.9999,0.0014,0.0005,1.5120
dt,Decision Tree Regressor,4.7984,204.8849,13.9533,0.9998,0.0018,0.0007,0.0850
lightgbm,Light Gradient Boosting Machine,12.1404,364.6940,19.0216,0.9997,0.0025,0.0016,0.1860
gbr,Gradient Boosting Regressor,17.5457,616.2083,24.7882,0.9995,0.0033,0.0024,0.6410
huber,Huber Regressor,23.4584,1168.2449,34.1439,0.9990,0.0043,0.0031,0.3730
par,Passive Aggressive Regressor,28.7687,1426.1505,37.5555,0.9988,0.0049,0.0039,0.5030
ada,AdaBoost Regressor,52.7692,4863.8066,69.6933,0.9958,0.0093,0.0072,0.3540
dummy,Dummy Regressor,870.0064,1158167.7093,1075.9946,-0.0012,0.1494,0.1230,0.0540


In [19]:
# Ignoruje kolumny z tempem, ponieważ nie mają ścisłe powiązanie z czasem
exp = setup(
    data=cleaned_df,
    target='Czas', # Czas ukończenia biegu w sekundach jako zmienna docelowa
    ignore_features=['Tempo', '5 km Tempo', '10 km Tempo', '15 km Tempo', '20 km Tempo','5 km Czas','10 km Czas','15 km Czas','20 km Czas'],
    keep_features=['Wiek', 'Płeć'],
    session_id=123,
    verbose=True,
    
)
# exp.dataset_transformed.sample(10)


,Description,Value
0,Session id,123
1,Target,Czas
2,Target type,Regression
3,Original data shape,"(17626, 25)"
4,Transformed data shape,"(17626, 28)"
5,Transformed train set shape,"(12338, 28)"
6,Transformed test set shape,"(5288, 28)"
7,Ignore features,9
8,Numeric features,10
9,Categorical features,5


In [20]:
best_model_v3 = exp.compare_models(sort='MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
dt,Decision Tree Regressor,28.2331,10228.0775,100.8051,0.9912,0.0124,0.0038,0.0690
rf,Random Forest Regressor,30.6800,3612.9383,59.6381,0.9969,0.0074,0.0041,1.2490
et,Extra Trees Regressor,33.3854,4339.9367,65.7410,0.9962,0.0080,0.0044,0.6750
lightgbm,Light Gradient Boosting Machine,41.5594,4232.0995,64.8708,0.9963,0.0081,0.0055,0.1780
gbr,Gradient Boosting Regressor,84.6336,16175.7127,127.0770,0.9860,0.0155,0.0109,0.5320
ada,AdaBoost Regressor,162.8559,47856.7350,218.7102,0.9586,0.0273,0.0212,0.2650
huber,Huber Regressor,192.3926,79500.2259,281.8762,0.9313,0.0384,0.0262,0.3550
par,Passive Aggressive Regressor,266.0228,120997.9046,341.5665,0.8951,0.0464,0.0364,0.4770
dummy,Dummy Regressor,870.0064,1158167.7093,1075.9946,-0.0012,0.1494,0.1230,0.0500


In [21]:
# Ignoruje kolumny z tempem, ponieważ nie mają ścisłe powiązanie z czasem, oraz sprawdzam różne dystanse czasu
exp = setup(
    data=cleaned_df,
    target='Czas', # Czas ukończenia biegu w sekundach jako zmienna docelowa
    ignore_features=['Tempo', '5 km Tempo', '10 km Tempo', '15 km Tempo', '20 km Tempo','10 km Czas','15 km Czas', '20 km Czas'],
    keep_features=['5 km Czas','Wiek','Płeć'],
    session_id=123,
    verbose=True,
    
)
# exp.dataset_transformed.sample(10)


,Description,Value
0,Session id,123
1,Target,Czas
2,Target type,Regression
3,Original data shape,"(17626, 25)"
4,Transformed data shape,"(17626, 29)"
5,Transformed train set shape,"(12338, 29)"
6,Transformed test set shape,"(5288, 29)"
7,Ignore features,8
8,Numeric features,11
9,Categorical features,5


In [22]:

best_model_v4 = exp.compare_models(sort='MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,18.4176,827.6158,28.7160,0.9993,0.0038,0.0026,0.7030
rf,Random Forest Regressor,26.9318,2277.1038,47.4767,0.9980,0.0061,0.0036,1.3330
dt,Decision Tree Regressor,27.9701,7801.1604,87.8099,0.9932,0.0110,0.0037,0.0710
lightgbm,Light Gradient Boosting Machine,31.7205,2093.0274,45.7006,0.9982,0.0059,0.0043,0.1780
gbr,Gradient Boosting Regressor,55.1459,5288.9949,72.7031,0.9954,0.0092,0.0073,0.5450
huber,Huber Regressor,97.1289,21472.0059,146.3865,0.9814,0.0183,0.0128,0.3630
ada,AdaBoost Regressor,138.6206,33789.4360,183.7833,0.9708,0.0232,0.0181,0.2770
par,Passive Aggressive Regressor,147.6776,38771.9133,190.9279,0.9665,0.0246,0.0198,0.4960
dummy,Dummy Regressor,870.0064,1158167.7093,1075.9946,-0.0012,0.1494,0.1230,0.0490


In [23]:
# Ignoruje kolumny z tempem, ponieważ nie mają ścisłe powiązanie z czasem, oraz sprawdzam różne dystansy czasu
exp = setup(
    data=cleaned_df,
    target='Czas', # Czas ukończenia biegu w sekundach jako zmienna docelowa
    ignore_features=['Tempo', '5 km Tempo', '10 km Tempo', '15 km Tempo', '20 km Tempo','15 km Czas', '20 km Czas'],
    keep_features=['10 km Czas', '5 km Czas','Wiek','Płeć'],
    session_id=123,
    verbose=True,
    
)
# exp.dataset_transformed.sample(10)


,Description,Value
0,Session id,123
1,Target,Czas
2,Target type,Regression
3,Original data shape,"(17626, 25)"
4,Transformed data shape,"(17626, 30)"
5,Transformed train set shape,"(12338, 30)"
6,Transformed test set shape,"(5288, 30)"
7,Ignore features,7
8,Numeric features,12
9,Categorical features,5


In [24]:
best_model_v5 = exp.compare_models(sort='MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,11.6598,361.1277,18.9201,0.9997,0.0026,0.0017,0.7350
rf,Random Forest Regressor,16.5644,722.1398,26.8244,0.9994,0.0036,0.0023,1.4210
dt,Decision Tree Regressor,18.4394,2329.3348,48.1160,0.9980,0.0064,0.0025,0.0730
lightgbm,Light Gradient Boosting Machine,21.7142,985.2137,31.3606,0.9991,0.0041,0.0029,0.1870
gbr,Gradient Boosting Regressor,36.9624,2156.4155,46.4197,0.9981,0.0061,0.0050,0.5800
huber,Huber Regressor,64.8831,10738.9192,103.5334,0.9907,0.0130,0.0085,0.3630
par,Passive Aggressive Regressor,80.8011,13019.7096,113.1293,0.9888,0.0146,0.0109,0.4950
ada,AdaBoost Regressor,117.9124,24081.0889,155.1355,0.9791,0.0200,0.0157,0.2930
dummy,Dummy Regressor,870.0064,1158167.7093,1075.9946,-0.0012,0.1494,0.1230,0.0520


In [25]:
# Ignoruje kolumny z tempem, ponieważ nie mają ścisłe powiązanie z czasem, oraz sprawdzam różne dystanse czasu
exp = setup(
    data=cleaned_df,
    target='Czas', # Czas ukończenia biegu w sekundach jako zmienna docelowa
    ignore_features=['Tempo', '5 km Tempo', '10 km Tempo', '15 km Tempo', '20 km Tempo', '20 km Czas'],
    keep_features=['15 km Czas', '10 km Czas', '5 km Czas','Wiek','Płeć'],
    session_id=123,
    verbose=True,
    
)
# exp.dataset_transformed.sample(10)


,Description,Value
0,Session id,123
1,Target,Czas
2,Target type,Regression
3,Original data shape,"(17626, 25)"
4,Transformed data shape,"(17626, 31)"
5,Transformed train set shape,"(12338, 31)"
6,Transformed test set shape,"(5288, 31)"
7,Ignore features,6
8,Numeric features,13
9,Categorical features,5


In [26]:
best_model_v6 = exp.compare_models(sort='MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,10.8587,348.7463,18.5810,0.9997,0.0025,0.0015,0.7780
rf,Random Forest Regressor,15.6124,711.5376,26.6180,0.9994,0.0035,0.0021,1.4780
dt,Decision Tree Regressor,18.8366,1944.1457,43.9002,0.9983,0.0057,0.0025,0.0750
lightgbm,Light Gradient Boosting Machine,21.7695,958.2847,30.9105,0.9992,0.0041,0.0029,0.1880
gbr,Gradient Boosting Regressor,35.1631,2065.0989,45.4072,0.9982,0.0060,0.0048,0.6160
huber,Huber Regressor,46.1227,5305.3914,72.7819,0.9954,0.0093,0.0061,0.3750
par,Passive Aggressive Regressor,54.7184,5701.8312,72.8461,0.9951,0.0092,0.0073,0.5140
ada,AdaBoost Regressor,98.7383,16417.5192,128.1030,0.9858,0.0167,0.0132,0.3110
dummy,Dummy Regressor,870.0064,1158167.7093,1075.9946,-0.0012,0.1494,0.1230,0.0520


In [27]:
best_model = compare_models([best_model_v1, best_model_v2, best_model_v3, best_model_v4, best_model_v5, best_model_v6])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
1,Extra Trees Regressor,10.8587,348.7463,18.5810,0.9997,0.0025,0.0015,0.7610
3,Extra Trees Regressor,10.8587,348.7463,18.5810,0.9997,0.0025,0.0015,0.7550
4,Extra Trees Regressor,10.8587,348.7463,18.5810,0.9997,0.0025,0.0015,0.7580
5,Extra Trees Regressor,10.8587,348.7463,18.5810,0.9997,0.0025,0.0015,0.7580
0,Random Forest Regressor,15.6124,711.5376,26.6180,0.9994,0.0035,0.0021,1.4660
2,Decision Tree Regressor,18.8366,1944.1457,43.9002,0.9983,0.0057,0.0025,0.0760


Wybieram najlepszy model z najmniejszym błędem

In [ ]:
# Strojenie hiperparametrów
best_model_tuned = exp.tune_model(
    estimator=best_model,  # Wybrany model
    n_iter=50,            # Liczba iteracji dla Random Grid Search
    optimize='MAE'         # Metryka do optymalizacji
)


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,10:26:21
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Extra Trees Regressor


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 50 candidates, totalling 500 fits


In [ ]:
best_final_model = compare_models([best_model, best_model_tuned])


## Porównanie oryginalnego modelu, oraz miodelu po strojeniu hiperparametrów wykazało że nie ma żadnej różnicy w podejściach.
### Możliwę że zwiększenie liczby iteracji by coś zmieniło, ale byłaby to marginalna zmiana

In [ ]:
exp.predict_model(best_final_model)


Podgląd predykcji pozwala zrozumieć, co model przewidywał i na podstawie jakich danych:

Cel Predykcji: Zmienna docelowa to prediction_label. Biorąc pod uwagę inne kolumny (5 km Czas Open, 10 km Czas Open, etc.), można ztwierdzić że model przewiduje czas biegu (np. półmaratonu/maratonu).

Kluczowe Predyktory: Model wykorzystuje szereg cech związanych z bieganiem, takich jak:

Czasy na krótszych dystansach (5 km Czas Open, 10 km Czas Open, 15 km Czas Open).

Płeć, Kategoria wiekowa (M20, K20, M30).

In [ ]:
plot_model(best_final_model, plot='feature')
plot_model(best_final_model, plot='error')
plot_model(best_final_model, plot='residuals')

In [ ]:
# Trenowanie modelu na całym zbiorze danych
final_model = finalize_model(best_final_model)

In [ ]:
# Zapis modelu lokalnie
final_model = finalize_model(best_model)
save_model(final_model, 'final_regression_pipeline')

In [ ]:
s3.upload_file(
    Filename='final_regression_pipeline.pkl',
    Bucket=BUCKET_NAME,
    Key='Train_Model/final_regression_pipeline.pkl'
)


In [ ]:
def predict_halfmarathon_time(wiek, plec, five_k_time_sec, model):
    """
    Funkcja do predykcji czasu ukończenia półmaratonu.

    Args:
        wiek (int): wiek biegacza
        plec (str): 'M' lub 'K'
        five_k_time_sec (int): czas 5 km w sekundach
        model: wytrenowany model pycaret

    Returns:
        str: czas w formacie H:M:S
    """
    # Dane wejściowe z wszystkimi kolumnami
    user_input = pd.DataFrame([{
        'Miejsce': 1.0,
        'Numer startowy': 1,
        'Imię': 'Test',
        'Nazwisko': 'User',
        'Kraj': 'POL',
        'Płeć': plec,
        'Płeć Miejsce': 1.0,
        'Kategoria wiekowa': 'M30' if plec == 'M' else 'K30',
        'Kategoria wiekowa Miejsce': 1.0,
        '5 km Czas': five_k_time_sec,
        '5 km Miejsce Open': 1.0,
        '10 km Czas': five_k_time_sec * 2,  # Przybliżenie
        '10 km Miejsce Open': 1.0,
        '15 km Czas': five_k_time_sec * 3,  # Przybliżenie
        '15 km Miejsce Open': 1.0,
        '20 km Czas': five_k_time_sec * 4,  # Przybliżenie
        '20 km Miejsce Open': 1.0,
        'Tempo Stabilność': 0.02,
        'Wiek': wiek
    }])

    # Predykcja
    prediction = predict_model(model, data=user_input)
    pred_seconds = int(round(prediction['prediction_label'][0]))

    # Konwersja na H:M:S
    pred_time_str = str(datetime.timedelta(seconds=pred_seconds))
    return pred_time_str

# Przykład użycia:
wiek_input = 42
plec_input = 'M'
czas_5km_input = 2000  # w sekundach

predykcja_czasu = predict_halfmarathon_time(wiek_input, plec_input, czas_5km_input, final_model)
print(f"Przewidywany czas ukończenia półmaratonu: {predykcja_czasu}")
